In [52]:
# Define sets of opcodes for different instruction types:
immediate_opcodes = {"ADDI", "ANDI", "ORI", "XORI", "SLTI", "SLL", "Scheduler.ipynb"}
load_opcodes      = {"LW"}
store_opcodes     = {"SW"}
branch_opcodes    = {"BEQ", "BNE"}
jump_opcodes      = {"J", "JR", "JAL"}

class Instruction:
    def __init__(self, line):
        # Remove commas and extra whitespace, then split into tokens.
        tokens = line.replace(",", "").split()
        if not tokens:
            return
        self.line = line.strip()
        # Set default fields
        self.opcode = tokens[0].upper() if line != "NOP"  else "NOP"
        self.dest = None
        self.rs = None
        self.rt = None
        self.immediate = None
        # Flags 
        self.is_load = False
        self.is_store = False
        self.is_branch = False
        self.is_jump = False

        # Parse based on opcode type:
        if self.opcode in load_opcodes:
            # Format: LW $t, offset($s)
            # tokens[1] is the destination register (t)
            # tokens[2] is in the form offset($s)
            self.dest = tokens[1]
            operand = tokens[2]
            open_paren = operand.find('(')
            close_paren = operand.find(')')
            if open_paren != -1 and close_paren != -1:
                self.immediate = operand[:open_paren]  # offset
                self.rs = operand[open_paren+1:close_paren]  # base register
            else:
                self.immediate = operand
            self.is_load = True

        elif self.opcode in store_opcodes:
            # Format: SW $t, offset($s)
            # For a store, there is no destination; 
            # we treat $t as the register whose value is stored.
            self.rt = tokens[1]
            operand = tokens[2]
            open_paren = operand.find('(')
            close_paren = operand.find(')')
            if open_paren != -1 and close_paren != -1:
                self.immediate = operand[:open_paren]  # offset
                self.rs = operand[open_paren+1:close_paren]  # base register
            else:
                self.immediate = operand
            self.is_store = True

        elif self.opcode in branch_opcodes:
            # Format: BEQ $s, $t, label
            # Branches only have two registers.
            self.rs = tokens[1]
            self.rt = tokens[2]
            if len(tokens) >= 4:
                self.immediate = tokens[3]  # branch target
            self.is_branch = True

        elif self.opcode in jump_opcodes:
            # Format: J label, JR $s, or JAL label
            if len(tokens) >= 2:
                self.immediate = tokens[1]
            self.is_jump = True

        elif self.opcode in immediate_opcodes:
            # Immediate instructions: e.g. ADDI $t, $s, imm
            # Here, destination is the first operand (t), then rs, then immediate.
            self.dest = tokens[1]
            self.rs = tokens[2]
            if len(tokens) >= 4:
                self.immediate = tokens[3]

        else:
            # Assume R-type instruction: e.g. ADD $d, $s, $t
            # Parsed as: dest = tokens[1], rs = tokens[2], rt = tokens[3]
            self.dest = tokens[1]
            self.rs = tokens[2]
            if len(tokens) >= 4:
                self.rt = tokens[3]

    def __str__(self):
        # Reconstruct a representation for debugging.
        parts = [self.opcode]
        if self.dest is not None:
            parts.append(self.dest)
        if self.rs is not None:
            parts.append(self.rs)
        if self.rt is not None:
            parts.append(self.rt)
        if self.immediate is not None:
            parts.append(self.immediate)
        return " ".join(parts)




In [ ]:
def unpseudo(instructions):
    unpseudoed = []
    for instruction in instructions.splitlines():
        label = ''
        
        instruction = instruction.replace(",", "").strip()
        instruction = instruction.split()
        if ":" in instruction[0]:
            label = str(instruction[0]) + ' '
            first = str(instruction[1]).upper()
            instruction = instruction[1:]
        else:
            first = str(instruction[0]).upper()
            
        if first == 'SGT':
            # Transform SGT: sgt A, B, C  =>  slt A, C, B
            unpseudoed.append(label + 'SLT ' + str(instruction[1]) + ' ' + str(instruction[3]) + ' ' + str(instruction[2]))
        elif first == 'LI':
            # Transform LI: li A, imm  =>  ori A, $0, imm
            unpseudoed.append(label + 'ORI ' + str(instruction[1]) + ' $0 ' + str(instruction[2]))
        elif first == 'BLT':
            unpseudoed.append(label + 'ORI $30 ' + str(instruction[1]) + ' ' + str(instruction[2]))
            unpseudoed.append('BNE $30 $0 ' + str(instruction[3]))
        elif first == 'BLE':
            unpseudoed.append(label + 'SLT $30 ' + str(instruction[2]) + ' ' + str(instruction[1]))
            unpseudoed.append('BEQ $30 $0 ' + str(instruction[3]))
        elif first == 'BGT':
            unpseudoed.append(label + 'SLT $30 ' + str(instruction[2]) + ' ' + str(instruction[1]))
            unpseudoed.append('BNE $30 $0 ' + str(instruction[3]))
        elif first == 'BGE':
            unpseudoed.append(label + 'SLT $30 ' + str(instruction[1]) + ' ' + str(instruction[2]))
            unpseudoed.append('BEQ $30 $0 ' + str(instruction[3]))
        elif first == 'BLTZ':
            unpseudoed.append(label + 'SLTI $30 ' + str(instruction[1]) + ' 0')
            unpseudoed.append('BNE $30 $0 ' + str(instruction[2]))
        elif first == 'BGEZ':
            unpseudoed.append(label + 'SLTI $30 ' + str(instruction[1]) + ' 0')
            unpseudoed.append('BEQ $30 $0 ' + str(instruction[2]))
        else:
            unpseudoed.append(label + ' '.join(instruction))
    return unpseudoed

def is_label(line):
    """Return True if the line is a label (ends with ':')."""
    return line.endswith(':')

def is_pseudo(line):
    """Returns if an instruction is pseudo or not"""
    tokens = line.split()
    if not tokens:
        return False
    opcode = tokens[0].upper()
    pseudos = {"SGT", "LI", "BLT", "BLE", "BGT", "BGE", "BLTZ", "BGEZ"}
    return opcode in pseudos

def is_branch(line):
    """Return True if the line starts with a branch opcode."""
    tokens = line.split()
    if not tokens:
        return False
    opcode = tokens[0].upper()
    branch_opcodes = {"BEQ", "BNE", "BLE", "BGT", "BLT", "BGE", "BLTZ", "BGEZ"}
    return opcode in branch_opcodes

def is_jump(line):
    """Return True if the line starts with a jump opcode."""
    tokens = line.split()
    if not tokens:
        return False
    opcode = tokens[0].upper()
    jump_opcodes = {"J", "JR", "JAL"}
    return opcode in jump_opcodes

def is_boundary(line):
    """A line is a boundary if it is a label, branch, or jump."""
    return is_label(line) or is_branch(line) or is_jump(line)

def segment_code(code_str):
    """
    Accepts a string containing the assembly code.
    - Removes comments (ignores any text following '#').
    - Splits the code into lines.
    - Segments the code so that boundaries (labels, branches, jumps)
      are isolated as their own segments.
      
    Returns a list of segments (each segment is a list of lines).
    """
    segments = []
    current_segment = []
    
    # Split the input string into individual lines.
    lines = code_str.splitlines()
    for raw_line in lines:
        # Remove comments (anything after '#' is discarded) and trim whitespace.
        line = raw_line.split('#')[0].strip()
        print(line)
        if not line:
            continue  # Skip empty lines
        
        # If the instruction is pseudo, convert it.
        if is_pseudo(line):
            unpseudo_lines = []
            unpseudo_lines = unpseudo(line)
        else:
            unpseudo_lines = []
            unpseudo_lines = [line]
            
        for line1 in unpseudo_lines:
            if is_boundary(line1):
                # Finalize any existing non-boundary segment.
                if current_segment:
                    # If the boundary isn't a label, add it to the current segment.
                    if not is_label(line1):
                        current_segment.append(line1)
                        segments.append(current_segment)
                    else:
                        segments.append(current_segment)
                        segments.append([line1])
                    
                    current_segment = []
                else:
                    # If there's no current segment, the boundary becomes its own segment.
                    segments.append([line1])
            else:
                # Non-boundary line: add to the current segment.
                current_segment.append(line1)
    
    # Add any leftover non-boundary instructions as a final segment.
    if current_segment:
        segments.append(current_segment)
    
    return segments

# Example usage with your assembly code snippet:
assembly_code = """\
main:
ORI $2 , $0, 0x0
ADDI $20, $0, 0xA
XORI $31, $0, 0x1
ANDI $5 , $0, 0x0
LW $10, 0x0($5)
LW $15, 0x0($5)
LOOP:
ADDI $2, $2, 1
SGT $25, $20, $2
BNE $25, $31, END
# Choose one of these Insertion based on your memory
# For Word addressable # For byte addressable
# ADD $5, $2, $0 # SLL $5, $2, 2
LW $16, 0x0($5)
SGT $26, $16, $10
BEQ $26, $0, MIN
OR $10, $16, $0
J LOOP
MIN:
SLT $27, $16, $15
BEQ $27, $0, LOOP
ADD $15, $16, $0
J LOOP
END:
NOP # (NOP equals to SLL $0, $0, 0)
"""

segments = segment_code(assembly_code)

# Display the resulting segments:
for idx, segment in enumerate(segments):
    print(f"Segment {idx}:")
    for line in segment:
        print(f"  {line}")
    print()


main:
ORI $2 , $0, 0x0
ADDI $20, $0, 0xA
XORI $31, $0, 0x1
ANDI $5 , $0, 0x0
LW $10, 0x0($5)
LW $15, 0x0($5)
LOOP:
ADDI $2, $2, 1
SGT $25, $20, $2
BNE $25, $31, END



SGT $26, $16, $10
LW $26, 0x0($5)
BEQ $26, $0, MIN
OR $10, $16, $0
J LOOP
MIN:
SLT $27, $16, $15
BEQ $27, $0, LOOP
ADD $15, $16, $0
J LOOP
END:
NOP
Segment 0:
  main:

Segment 1:
  ORI $2 , $0, 0x0
  ADDI $20, $0, 0xA
  XORI $31, $0, 0x1
  ANDI $5 , $0, 0x0
  LW $10, 0x0($5)
  LW $15, 0x0($5)

Segment 2:
  LOOP:

Segment 3:
  ADDI $2, $2, 1
  SLT $25 $2 $20
  BNE $25, $31, END

Segment 4:
  SLT $26 $10 $16
  LW $26, 0x0($5)
  BEQ $26, $0, MIN

Segment 5:
  OR $10, $16, $0
  J LOOP

Segment 6:
  MIN:

Segment 7:
  SLT $27, $16, $15
  BEQ $27, $0, LOOP

Segment 8:
  ADD $15, $16, $0
  J LOOP

Segment 9:
  END:

Segment 10:
  NOP



In [55]:
from collections import defaultdict, deque

def make_nop():
    nop = Instruction("SLL $0, $0, 0")
    nop.opcode = "NOP"
    nop.dest = None
    nop.rs = None
    nop.rt = None
    nop.immediate = None
    nop.is_branch = False
    nop.is_load = False
    return nop

def build_dependency_graph(instrs):
    graph = defaultdict(list)
    in_degree = {i: 0 for i in instrs}
    
    for i, inst_i in enumerate(instrs):
        if inst_i.dest is None:
            continue
        for j in range(i+1, len(instrs)):
            inst_j = instrs[j]
            sources = [inst_j.rs, inst_j.rt] if inst_j.rs or inst_j.rt else []
            if inst_i.dest in sources:
                graph[inst_i].append(inst_j)
                in_degree[inst_j] += 1
    return graph, in_degree

def schedule_instructions(instrs):
    graph, in_degree = build_dependency_graph(instrs)
    ready_list = deque([i for i in instrs if (in_degree[i] == 0 or i.is_jump)])
    scheduled = []

    while ready_list:
        slot1, slot2 = None, None
        ready = list(ready_list)
        non_branches_jumps = [i for i in ready if not i.is_branch and not i.is_jump]
        branches = [i for i in ready if i.is_branch]
        jumps = [i for i in ready if i.is_jump]
        

        if branches or jumps:
            slot2 = branches[0] if branches else jumps[0]
            ready_list.remove(slot2)
            if non_branches_jumps:
                slot1 = non_branches_jumps[0]
                ready_list.remove(slot1)
            else:
                slot1 = make_nop()
        else:
            if len(ready_list) >= 2:
                slot1 = ready_list.popleft()
                slot2 = ready_list.popleft()
            elif len(ready_list) == 1:
                slot1 = ready_list.popleft()
                slot2 = make_nop()
            else:
                break
        
        scheduled.append((slot1, slot2))
        
        if slot1.is_load and slot1.dest:
            sources = [slot2.rs, slot2.rt] if slot2 and slot2.opcode != "NOP" else []
            if slot1.dest in sources:
                scheduled.append(("SLL $0, $0, 0", "SLL $0, $0, 0"))
        for inst in [slot1, slot2]:
            if inst.opcode == "NOP":
                continue
            for succ in graph.get(inst, []):
                in_degree[succ] -= 1
                if in_degree[succ] == 0:
                    ready_list.append(succ)

    return [inst for packet in scheduled for inst in packet]

def schedule_segments(segments):
    final_code = []
    for segment in segments:
        if len(segment) == 1 and is_boundary(segment[0]):
            final_code.append(segment[0])
        elif len(segment) == 1 and segment[0] == 'NOP':
            final_code.append("SLL $0, $0, 0")
        else:
            instrs = [Instruction(inst) for inst in segment]
            scheduled_instrs = schedule_instructions(instrs)
            final_code.extend([inst.line for inst in scheduled_instrs])
    return "\n".join(final_code)

scheduled_code = schedule_segments(segments)
print(scheduled_code)


main:
ORI $2 , $0, 0x0
ADDI $20, $0, 0xA
XORI $31, $0, 0x1
ANDI $5 , $0, 0x0
LW $10, 0x0($5)
LW $15, 0x0($5)
LOOP:
ADDI $2, $2, 1
SLL $0, $0, 0
SLT $25 $2 $20
SLL $0, $0, 0
SLL $0, $0, 0
BNE $25, $31, END
SLT $26 $10 $16
LW $26, 0x0($5)
SLL $0, $0, 0
BEQ $26, $0, MIN
OR $10, $16, $0
J LOOP
MIN:
SLT $27, $16, $15
SLL $0, $0, 0
SLL $0, $0, 0
BEQ $27, $0, LOOP
ADD $15, $16, $0
J LOOP
END:
SLL $0, $0, 0
